In [33]:
import pandas as pd
import numpy as np

In [34]:
df = pd.read_csv('iris.csv')

In [35]:
df

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [36]:
df.isnull().sum()

sepal_length    0
sepal_width     0
petal_length    0
petal_width     0
species         0
dtype: int64

In [37]:
import json

In [38]:
with open('file.json', 'r') as json_file:
    data = json.load(json_file)

In [39]:
data

{'session_name': 'test',
 'session_description': 'test',
 'design_state_data': {'session_info': {'project_id': '1',
   'experiment_id': 'kkkk-11',
   'dataset': 'iris_modified.csv',
   'session_name': 'test',
   'session_description': 'test'},
  'target': {'prediction_type': 'Regression',
   'target': 'petal_width',
   'type': 'regression',
   'partitioning': True},
  'train': {'policy': 'Split the dataset',
   'time_variable': 'sepal_length',
   'sampling_method': 'No sampling(whole data)',
   'split': 'Randomly',
   'k_fold': False,
   'train_ratio': 0,
   'random_seed': 0},
  'metrics': {'optomize_model_hyperparameters_for': 'AUC',
   'optimize_threshold_for': 'F1 Score',
   'compute_lift_at': 0,
   'cost_matrix_gain_for_true_prediction_true_result': 1,
   'cost_matrix_gain_for_true_prediction_false_result': 0,
   'cost_matrix_gain_for_false_prediction_true_result': 0,
   'cost_matrix_gain_for_false_prediction_false_result': 0},
  'feature_handling': {'sepal_length': {'feature_name'

In [40]:
print(data['design_state_data']['target'])

{'prediction_type': 'Regression', 'target': 'petal_width', 'type': 'regression', 'partitioning': True}


In [41]:
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv('iris.csv')

# Get the column names
features = df.columns

for feature in features:
    missing_values = df[feature].isnull().sum()
    print("Missing Value Count for",feature,' : ',missing_values)

Missing Value Count for sepal_length  :  0
Missing Value Count for sepal_width  :  0
Missing Value Count for petal_length  :  0
Missing Value Count for petal_width  :  0
Missing Value Count for species  :  0


In [43]:
# As there are no missing values in our dataset we won't be running this block

df['sepal_length'] = df['sepal_length'].fillna(df['sepal_length'].mean())

In [44]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel

def reduce_features(df, method, target, **kwargs):
    if method == "No Reduction":
        return df
    elif method == "Corr with Target":
        corr = df.corr()[target]
        correlated_features = corr[np.abs(corr) > kwargs["threshold"]].index
        return df[correlated_features]
    elif method == "Tree-based":
        X = df.drop(target, axis=1)
        y = df[target]
        clf = RandomForestClassifier(random_state=0)
        sfm = SelectFromModel(clf, threshold=kwargs["threshold"])
        sfm.fit(X, y)
        return X.columns[sfm.get_support()]
    elif method == "PCA":
        target = 'species'
        X = df.drop(target, axis=1)
        y = df[target]
        pca = PCA(n_components=kwargs["n_components"])
        pca.fit(X)
        return pca.transform(X)
    else:
        raise ValueError("Invalid method selected")


In [45]:
# Example usage:
Iris = pd.read_csv("iris.csv")
# df = df.drop(df.columns[-1], axis=1)
reduced_data = reduce_features(Iris, "PCA", "species", n_components=0.9)

In [46]:
for i in data['design_state_data']['algorithms']:
  print(i)

RandomForestClassifier
RandomForestRegressor
GBTClassifier
GBTRegressor
LinearRegression
LogisticRegression
RidgeRegression
LassoRegression
ElasticNetRegression
xg_boost
DecisionTreeRegressor
DecisionTreeClassifier
SVM
SGD
KNN
extra_random_trees
neural_network


In [26]:
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

In [27]:
# Get the prediction_type from JSON
prediction_type = data['design_state_data']['target']['prediction_type'].lower()

if prediction_type == "classification":
    model = LogisticRegression()
elif prediction_type == "regression":
    model = LinearRegression()
    print("Linear Regression ")
elif prediction_type == "clustering":
    model = KNeighborsClassifier()
else:
    raise ValueError("Invalid prediction_type specified in JSON.")

Linear Regression 


In [47]:
X = df[["sepal_length","sepal_width","petal_length","petal_width"]]
y = df["species"]

In [48]:
import json
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

# Define the parameter grid
param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}

# Create an instance of SVC
svc = SVC()

# Create the grid search object
grid_search = GridSearchCV(svc, param_grid, cv=5)

# Fit the grid search to the data
grid_search.fit(X, y)

# Save the best parameters to a JSON file
best_params = grid_search.best_params_
with open('best_params.json', 'w') as f:
    json.dump(best_params, f)

In [50]:
# Grid search cross validation
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
grid_search={"C":np.logspace(-3,3,7), "penalty":["l1","l2"]}# l1 lasso l2 ridge
log=LogisticRegression()
log_cv=GridSearchCV(log,grid_search,cv=10)
log_cv.fit(X,y)

print("tuned hpyerparameters :(best parameters) ",log_cv.best_params_)
print("accuracy :",log_cv.best_score_)

tuned hpyerparameters :(best parameters)  {'C': 10.0, 'penalty': 'l2'}
accuracy : 0.9866666666666667
